In [1]:
import pandas as pd
import numpy as np
import json
from tqdm.notebook import tqdm

In [2]:
#最大表示列数の指定（ここでは50列を指定）
pd.set_option('display.max_columns', 70)
pd.set_option('display.max_rows', 100)

In [3]:
path_input = {}
path_output = {}
for i in ["train2","train1", "valid"]:
    path_input[i] = f"/kaggle/input/otto-make-data-train-test-split/{i}.jsonl"
    path_output[i] = f"aid_summary_{i}.pickle"
    
sample_size = 150000

In [4]:
file = {}
# 出力先ファイル
for i in ["train1", "valid"]:
    file[i] = open(path_output[i], "w")

In [5]:
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

In [6]:
# train-all と valid-sampleの上位aid
for i in ["train1", "valid"]:
    if i == "valid": lastweek = "train1"
    elif i == "train1": lastweek = "train2"
    dict_count = {"clicks": {}, "carts": {}, "orders": {}, "all": {}}
    dict_count_lastweek = {"clicks": {}, "carts": {}, "orders": {}, "all": {}}
    dict_count2 = {"clicks2clicks": {}, "clicks2carts": {}, "clicks2orders": {}, 
                   "carts2clicks": {}, "carts2carts": {}, "carts2orders": {},
                   "orders2clicks": {}, "orders2carts": {}, "orders2orders": {}}
    dict_count2_lastweek = {"clicks2clicks": {}, "clicks2carts": {}, "clicks2orders": {}, 
                   "carts2clicks": {}, "carts2carts": {}, "carts2orders": {},
                   "orders2clicks": {}, "orders2carts": {}, "orders2orders": {}}
    # 今週
    chunks = pd.read_json(path_input[i],lines = True, chunksize = sample_size)
    for df_chunk in tqdm(chunks):
        for j, session in df_chunk.iterrows():
            # train, valid共にcutoff以前のデータのみで特徴量を集計する
            event_train = session["events"][:session["cutoff"]]
            set_uniq = {"clicks": set(), "carts": set(), "orders": set(), "all": set()}
            set_uniq2 = {"clicks2clicks": set(), "clicks2carts": set(), "clicks2orders": set(),
                         "carts2clicks": set(), "carts2carts": set(), "carts2orders": set(),
                         "orders2clicks": set(), "orders2carts": set(), "orders2orders": set()} # 2回
            for event in event_train:
                aid = event["aid"]
                types = event["type"]
                for pretypes in ["clicks", "carts", "orders"]:
                    if aid in set_uniq[pretypes] and aid not in set_uniq2[f"{pretypes}2{types}"]:
                        dict_count2[f"{pretypes}2{types}"][aid] = dict_count2[f"{pretypes}2{types}"].get(aid, 0) + 1
                        set_uniq2[f"{pretypes}2{types}"].add(aid)
                if aid not in set_uniq[types]:
                    dict_count[types][aid] = dict_count[types].get(aid, 0) + 1
                    set_uniq[types].add(aid)
                if aid not in set_uniq["all"]:
                    dict_count["all"][aid] = dict_count["all"].get(aid, 0) + 1
                    set_uniq["all"].add(aid)
            
    df_count = pd.DataFrame(dict_count).reset_index()
    df_count.columns = ["aid", "n_clicks_allses", "n_carts_allses", "n_orders_allses", "n_alltypes_allses"]
    df_count2 = pd.DataFrame(dict_count2).reset_index()
    df_count2.columns = ["aid", "n_clicks2clicks_allses", "n_clicks2carts_allses", "n_clicks2orders_allses",
                         "n_carts2clicks_allses", "n_carts2carts_allses", "n_carts2orders_allses",
                         "n_orders2clicks_allses", "n_orders2carts_allses", "n_orders2orders_allses"]
    # 先週
    chunks = pd.read_json(path_input[lastweek],lines = True, chunksize = sample_size)
    for df_chunk in tqdm(chunks):
        for j, session in df_chunk.iterrows():
            # 全期間を集計
            event_train = session["events"]
            set_uniq = {"clicks": set(), "carts": set(), "orders": set(), "all": set()}
            set_uniq2 = {"clicks2clicks": set(), "clicks2carts": set(), "clicks2orders": set(),
                         "carts2clicks": set(), "carts2carts": set(), "carts2orders": set(),
                         "orders2clicks": set(), "orders2carts": set(), "orders2orders": set()} # 2回
            for event in event_train:
                aid = event["aid"]
                types = event["type"]
                for pretypes in ["clicks", "carts", "orders"]:
                    if aid in set_uniq[pretypes] and aid not in set_uniq2[f"{pretypes}2{types}"]:
                        dict_count2_lastweek[f"{pretypes}2{types}"][aid] = dict_count2_lastweek[f"{pretypes}2{types}"].get(aid, 0) + 1
                        set_uniq2[f"{pretypes}2{types}"].add(aid)
                if aid not in set_uniq[types]:
                    dict_count_lastweek[types][aid] = dict_count_lastweek[types].get(aid, 0) + 1
                    set_uniq[types].add(aid)
                if aid not in set_uniq["all"]:
                    dict_count_lastweek["all"][aid] = dict_count_lastweek["all"].get(aid, 0) + 1
                    set_uniq["all"].add(aid)
            
    df_count_lastweek = pd.DataFrame(dict_count_lastweek).reset_index()
    df_count_lastweek.columns = ["aid", "n_clicks_allses_lastweek", "n_carts_allses_lastweek", "n_orders_allses_lastweek", "n_alltypes_allses_lastweek"]
    df_count2_lastweek = pd.DataFrame(dict_count2_lastweek).reset_index()
    df_count2_lastweek.columns = ["aid", "n_clicks2clicks_allses_lastweek", "n_clicks2carts_allses_lastweek", "n_clicks2orders_allses_lastweek",
                         "n_carts2clicks_allses_lastweek", "n_carts2carts_allses_lastweek", "n_carts2orders_allses_lastweek",
                         "n_orders2clicks_allses_lastweek", "n_orders2carts_allses_lastweek", "n_orders2orders_allses_lastweek"]

    # 先週と今週をマージ
    df_count = pd.merge(df_count, df_count_lastweek, how = "outer", on = "aid")
    df_count = pd.merge(df_count, df_count2, how = "outer", on = "aid")
    df_count = pd.merge(df_count, df_count2_lastweek, how = "outer", on = "aid")
    df_count = df_count.fillna(0).astype(np.int32)
    for types in ["clicks", "carts", "orders", "alltypes"]:
        df_count[f"n_{types}_allses_diff"] = df_count[f"n_{types}_allses"] - df_count[f"n_{types}_allses_lastweek"]
        df_count[f"n_{types}_allses_diffrate"] = df_count[f"n_{types}_allses"] / (df_count[f"n_{types}_allses_lastweek"]+0.5)
    for pretypes in ["clicks", "carts", "orders"]:
        for types in ["clicks", "carts", "orders"]:
            df_count[f"n_{pretypes}2{types}_allses_lastweek"] = df_count[f"n_{pretypes}2{types}_allses"] + df_count[f"n_{pretypes}2{types}_allses_lastweek"]
            df_count[f"rate_{pretypes}2{types}"] = df_count[f"n_{pretypes}2{types}_allses"] / (df_count[f"n_{pretypes}_allses"]+0.5)
            df_count[f"rate_{pretypes}2{types}_lastweek"] = df_count[f"n_{pretypes}2{types}_allses_lastweek"] / (df_count[f"n_{pretypes}_allses_lastweek"]+0.5)

    df_count = reduce_mem_usage(df_count)
    display(df_count)
    df_count.to_pickle(path_output[i])

0it [00:00, ?it/s]

0it [00:00, ?it/s]

Mem. usage decreased to 156.40 Mb (64.6% reduction)


,aid,n_clicks_allses,n_carts_allses,n_orders_allses,n_alltypes_allses,n_clicks_allses_lastweek,n_carts_allses_lastweek,n_orders_allses_lastweek,n_alltypes_allses_lastweek,n_clicks2clicks_allses,n_clicks2carts_allses,n_clicks2orders_allses,n_carts2clicks_allses,n_carts2carts_allses,n_carts2orders_allses,n_orders2clicks_allses,n_orders2carts_allses,n_orders2orders_allses,n_clicks2clicks_allses_lastweek,n_clicks2carts_allses_lastweek,n_clicks2orders_allses_lastweek,n_carts2clicks_allses_lastweek,n_carts2carts_allses_lastweek,n_carts2orders_allses_lastweek,n_orders2clicks_allses_lastweek,n_orders2carts_allses_lastweek,n_orders2orders_allses_lastweek,n_clicks_allses_diff,n_clicks_allses_diffrate,n_carts_allses_diff,n_carts_allses_diffrate,n_orders_allses_diff,n_orders_allses_diffrate,n_alltypes_allses_diff,n_alltypes_allses_diffrate,rate_clicks2clicks,rate_clicks2clicks_lastweek,rate_clicks2carts,rate_clicks2carts_lastweek,rate_clicks2orders,rate_clicks2orders_lastweek,rate_carts2clicks,rate_carts2clicks_lastweek,rate_carts2carts,rate_carts2carts_lastweek,rate_carts2orders,rate_carts2orders_lastweek,rate_orders2clicks,rate_orders2clicks_lastweek,rate_orders2carts,rate_orders2carts_lastweek,rate_orders2orders,rate_orders2orders_lastweek
0,573273,78,5,1,78,858,124,60,873,21,3,1,4,0,1,1,0,0,310,102,52,75,19,59,20,0,0,-780,0.090881,-119,0.040161,-59,0.016525,-795,0.089294,0.267578,0.361084,0.038208,0.118835,0.012741,0.060577,0.727051,0.602539,0.0,0.152588,0.181763,0.473877,0.666504,0.330566,0.0,0.0,0.0,0.000000
1,921137,7,0,0,7,5,1,0,6,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,1.272461,-1,0.000000,0,0.000000,1,1.077148,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000
2,930597,15,0,0,15,32,4,1,33,2,0,0,0,0,0,0,0,0,11,4,0,1,0,0,0,0,0,-17,0.461426,-4,0.000000,-1,0.000000,-18,0.447754,0.129028,0.338379,0.000000,0.123047,0.000000,0.000000,0.000000,0.222168,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000
3,1037630,22,1,0,22,40,0,0,40,5,1,0,1,0,0,0,0,0,14,1,0,1,0,0,0,0,0,-18,0.542969,1,2.000000,0,0.000000,-18,0.542969,0.222168,0.345703,0.044434,0.024689,0.000000,0.000000,0.666504,2.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000
4,756920,10,1,0,10,21,1,1,21,1,1,0,0,0,0,0,0,0,5,2,1,0,0,1,0,0,0,-11,0.465088,0,0.666504,-1,0.000000,-11,0.465088,0.095215,0.232544,0.095215,0.093018,0.000000,0.046509,0.000000,0.000000,0.0,0.000000,0.000000,0.666504,0.000000,0.000000,0.0,0.0,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1504512,130079,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.000000,0,0.000000,-1,0.000000,-1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000
1504513,625382,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.000000,0,0.000000,-1,0.000000,-1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000
1504514,416291,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0.000000,0,0.000000,-1,0.000000,-1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.666504
1504515,1525088,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.000000,0,0.000000,-1,0.000000,-1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000


0it [00:00, ?it/s]

0it [00:00, ?it/s]

Mem. usage decreased to 147.85 Mb (63.6% reduction)


,aid,n_clicks_allses,n_carts_allses,n_orders_allses,n_alltypes_allses,n_clicks_allses_lastweek,n_carts_allses_lastweek,n_orders_allses_lastweek,n_alltypes_allses_lastweek,n_clicks2clicks_allses,n_clicks2carts_allses,n_clicks2orders_allses,n_carts2clicks_allses,n_carts2carts_allses,n_carts2orders_allses,n_orders2clicks_allses,n_orders2carts_allses,n_orders2orders_allses,n_clicks2clicks_allses_lastweek,n_clicks2carts_allses_lastweek,n_clicks2orders_allses_lastweek,n_carts2clicks_allses_lastweek,n_carts2carts_allses_lastweek,n_carts2orders_allses_lastweek,n_orders2clicks_allses_lastweek,n_orders2carts_allses_lastweek,n_orders2orders_allses_lastweek,n_clicks_allses_diff,n_clicks_allses_diffrate,n_carts_allses_diff,n_carts_allses_diffrate,n_orders_allses_diff,n_orders_allses_diffrate,n_alltypes_allses_diff,n_alltypes_allses_diffrate,rate_clicks2clicks,rate_clicks2clicks_lastweek,rate_clicks2carts,rate_clicks2carts_lastweek,rate_clicks2orders,rate_clicks2orders_lastweek,rate_carts2clicks,rate_carts2clicks_lastweek,rate_carts2carts,rate_carts2carts_lastweek,rate_carts2orders,rate_carts2orders_lastweek,rate_orders2clicks,rate_orders2clicks_lastweek,rate_orders2carts,rate_orders2carts_lastweek,rate_orders2orders,rate_orders2orders_lastweek
0,11830,854,126,15,861,1730,248,105,1754,178,118,12,40,15,13,1,0,1,588,350,102,140,52,99,10,3,15,-876,0.493408,-122,0.506836,-90,0.142212,-893,0.490723,0.208252,0.339844,0.138062,0.202271,0.014046,0.058929,0.316162,0.563477,0.118591,0.209229,0.102783,0.398438,0.064514,0.094788,0.0,0.028442,0.064514,0.142212
1,1679529,3,1,0,3,28,3,0,28,1,1,0,0,1,0,0,0,0,3,4,0,1,1,0,0,0,0,-25,0.105286,-2,0.285645,0,0.000000,-25,0.105286,0.285645,0.105286,0.285645,0.140381,0.000000,0.000000,0.000000,0.285645,0.666504,0.285645,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000
2,92401,22,7,0,23,42,9,7,43,1,6,0,1,1,0,0,0,0,9,14,6,4,2,6,0,0,1,-20,0.517578,-2,0.736816,-7,0.000000,-20,0.528809,0.044434,0.211792,0.266602,0.329346,0.000000,0.141235,0.133301,0.421143,0.133301,0.210571,0.000000,0.631348,0.000000,0.000000,0.0,0.000000,0.000000,0.133301
3,1055218,64,8,2,65,157,20,9,159,10,8,1,2,2,1,0,0,0,40,27,9,12,5,9,3,0,0,-93,0.406250,-12,0.390137,-7,0.210571,-94,0.407471,0.155029,0.253906,0.124023,0.171387,0.015503,0.057129,0.235352,0.585449,0.235352,0.243896,0.117676,0.438965,0.000000,0.315674,0.0,0.000000,0.000000,0.000000
4,1561739,7,2,0,7,7,2,2,8,2,2,0,1,1,0,0,0,0,4,4,1,2,1,1,0,0,0,0,0.933105,0,0.799805,-2,0.000000,-1,0.823730,0.266602,0.533203,0.266602,0.533203,0.000000,0.133301,0.399902,0.799805,0.399902,0.399902,0.000000,0.399902,0.000000,0.000000,0.0,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1384229,557875,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.000000,0,0.000000,-1,0.000000,-1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000
1384230,1605760,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.000000,0,0.000000,-1,0.000000,-1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000
1384231,1157861,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.000000,0,0.000000,-1,0.000000,-1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000
1384232,620121,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.000000,0,0.000000,-1,0.000000,-1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000
